In [1]:
!pip install pandas numpy scikit-learn

import pandas as pd
import numpy as np
import sqlite3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


In [2]:
from google.colab import files
uploaded = files.upload()

df = pd.read_csv(list(uploaded.keys())[0])

# Standardize column names
df.columns = df.columns.str.strip()
df.rename(columns={
    "Customer ID": "CustomerID",
    "Price": "UnitPrice",
    "Invoice": "InvoiceNo"
}, inplace=True)

# Convert date
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# Monetary column
df["transaction_value"] = df["Quantity"] * df["UnitPrice"]

# Cleaning
df = df[df["Quantity"] > 0]
df = df.dropna(subset=["CustomerID"])
df = df.drop_duplicates().reset_index(drop=True)

df.head()


Saving online_retail_II.csv to online_retail_II.csv


InvoiceNo StockCode                          Description  Quantity  \
0    489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1    489434    79323P                   PINK CHERRY LIGHTS        12   
2    489434    79323W                  WHITE CHERRY LIGHTS        12   
3    489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4    489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  UnitPrice  CustomerID         Country  \
0 2009-12-01 07:45:00       6.95     13085.0  United Kingdom   
1 2009-12-01 07:45:00       6.75     13085.0  United Kingdom   
2 2009-12-01 07:45:00       6.75     13085.0  United Kingdom   
3 2009-12-01 07:45:00       2.10     13085.0  United Kingdom   
4 2009-12-01 07:45:00       1.25     13085.0  United Kingdom   

   transaction_value  
0               83.4  
1               81.0  
2               81.0  
3              100.8  
4               30.0

In [3]:
products_df = df[["StockCode","Description","UnitPrice"]].drop_duplicates()

products_df.rename(columns={
    "StockCode":"product_id",
    "Description":"product_name",
    "UnitPrice":"unit_price"
}, inplace=True)

texts = products_df["product_name"].fillna("")

vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(texts)

kmeans = KMeans(n_clusters=6, random_state=42)
products_df["cluster_id"] = kmeans.fit_predict(X)

label_map = {
    0: "Home Decor",
    1: "Accessories",
    2: "Toys & Gifts",
    3: "Kitchen Items",
    4: "Stationery",
    5: "Miscellaneous"
}

products_df["product_category"] = products_df["cluster_id"].map(label_map)
products_df.drop(columns=["cluster_id"], inplace=True)


In [4]:
customers_df = df.groupby("CustomerID").agg({
    "InvoiceDate":"max",
    "Country":"first"
}).reset_index()

customers_df.rename(columns={
    "CustomerID":"customer_id",
    "InvoiceDate":"last_purchase_date",
    "Country":"country"
}, inplace=True)

customers_df["email"] = customers_df["customer_id"].astype(str)+"@mail.com"
customers_df["loyalty_status"] = np.random.choice(["Bronze","Silver","Gold"], len(customers_df))
customers_df["total_loyalty_points"] = np.random.randint(100,5000,len(customers_df))


In [5]:
stores_df = df[["Country"]].drop_duplicates().reset_index(drop=True)

stores_df["store_id"] = stores_df.index + 1
stores_df["store_name"] = "Store_" + stores_df["Country"]
stores_df["store_region"] = stores_df["Country"]


In [6]:
sales_header_df = df.groupby("InvoiceNo").agg({
    "CustomerID":"first",
    "InvoiceDate":"first",
    "transaction_value":"sum"
}).reset_index()

sales_header_df.rename(columns={
    "InvoiceNo":"transaction_id",
    "CustomerID":"customer_id",
    "InvoiceDate":"transaction_date",
    "transaction_value":"total_amount"
}, inplace=True)

sales_header_df["customer_id"] = sales_header_df["customer_id"].astype(int)


In [7]:
sales_line_items_df = df.copy()

sales_line_items_df.rename(columns={
    "InvoiceNo":"transaction_id",
    "StockCode":"product_id",
    "Quantity":"quantity"
}, inplace=True)

sales_line_items_df["line_item_amount"] = sales_line_items_df["transaction_value"]
sales_line_items_df["line_item_id"] = sales_line_items_df.index + 1

sales_line_items_df = sales_line_items_df[[
    "line_item_id","transaction_id","product_id","quantity","line_item_amount"
]]


In [8]:
products_df.to_csv("products.csv",index=False)
customers_df.to_csv("customers.csv",index=False)
stores_df.to_csv("stores.csv",index=False)
sales_header_df.to_csv("sales_header.csv",index=False)
sales_line_items_df.to_csv("sales_line_items.csv",index=False)


In [9]:
conn = sqlite3.connect("retail.db")

products_df.to_sql("products",conn,if_exists="replace",index=False)
customers_df.to_sql("customers",conn,if_exists="replace",index=False)
stores_df.to_sql("stores",conn,if_exists="replace",index=False)
sales_header_df.to_sql("sales_header",conn,if_exists="replace",index=False)
sales_line_items_df.to_sql("sales_line_items",conn,if_exists="replace",index=False)


779495

In [10]:
query = """
SELECT
    sh.customer_id,
    COUNT(sh.transaction_id) AS frequency,
    SUM(sh.total_amount) AS monetary,
    AVG(sh.total_amount) AS avg_spend,
    SUM(sli.quantity) AS total_products,
    MAX(sh.transaction_date) AS last_purchase_date
FROM sales_header sh
JOIN sales_line_items sli
ON sh.transaction_id = sli.transaction_id
GROUP BY sh.customer_id
"""

features_df = pd.read_sql(query, conn)

features_df["last_purchase_date"] = pd.to_datetime(features_df["last_purchase_date"])
snapshot = features_df["last_purchase_date"].max() + pd.Timedelta(days=1)

features_df["recency"] = (snapshot - features_df["last_purchase_date"]).dt.days
features_df.drop(columns=["last_purchase_date"], inplace=True)

# Demo Target
features_df["target_spend"] = features_df["monetary"] * 0.3


In [11]:
features_df.to_csv("ml_dataset.csv",index=False)
features_df.to_sql("ml_features",conn,if_exists="replace",index=False)


5881

In [12]:
bad_records = []

for idx, row in df.iterrows():
    reason = []

    if row["Quantity"] <= 0:
        reason.append("Invalid Quantity")

    if row["UnitPrice"] <= 0:
        reason.append("Invalid Price")

    if pd.isna(row["CustomerID"]):
        reason.append("Missing CustomerID")

    if pd.isna(row["InvoiceNo"]):
        reason.append("Missing Invoice")

    if row["transaction_value"] <= 0:
        reason.append("Invalid Transaction Value")

    if reason:
        bad_row = row.copy()
        bad_row["reason_flag"] = ", ".join(reason)
        bad_records.append(bad_row)

bad_records_df = pd.DataFrame(bad_records)


In [13]:
df_clean = df.drop(bad_records_df.index)


In [14]:
bad_records_df.to_csv("bad_records.csv", index=False)
